# 🔧 WSPlumber - Backtest Completo en Colab

Este notebook ejecuta el sistema completo de WSPlumber (Mains + Hedges + Recoveries) con la optimización de skip de ticks vacíos.

**Requisitos:**
- Subir el archivo CSV de datos M1
- Runtime con suficiente RAM (Standard está bien)

## 1. Instalación y Setup

In [ ]:
# Clonar el repositorio (ajustar URL si es privado)
!git clone https://github.com/asanchezmanas/wsplumber.git
%cd wsplumber

# Instalar dependencias
!pip install -r requirements.txt --quiet
!pip install polars structlog --quiet

print("✅ WSPlumber instalado correctamente")

In [ ]:
# Subir archivo de datos
from google.colab import files

print("📁 Sube tu archivo CSV de datos M1:")
uploaded = files.upload()

# Obtener nombre del archivo subido
csv_filename = list(uploaded.keys())[0]
print(f"\n✅ Archivo cargado: {csv_filename}")

## 2. Configuración del Backtest

In [ ]:
# Configuración
import sys
sys.path.insert(0, 'src')

# Parámetros del backtest
INITIAL_BALANCE = 10000.0  # Balance inicial en EUR
MAX_TICKS = 100_000        # Limitar ticks para test rápido (None = todos)
PAIR = "EURUSD"

print(f"📊 Configuración:")
print(f"   Balance Inicial: {INITIAL_BALANCE:,.2f} EUR")
print(f"   Max Ticks: {MAX_TICKS if MAX_TICKS else 'Todos'}")
print(f"   Par: {PAIR}")

## 3. Ejecutar Backtest

In [ ]:
import asyncio
import time

from wsplumber.core.backtest.backtest_engine import BacktestEngine

async def run_backtest():
    print("🚀 Iniciando backtest...")
    print("="*60)
    
    start = time.time()
    
    engine = BacktestEngine(initial_balance=INITIAL_BALANCE)
    
    await engine.run(
        csv_path=csv_filename,
        pair=PAIR,
        audit_file='backtest_audit.md',
        max_bars=MAX_TICKS
    )
    
    elapsed = time.time() - start
    print(f"\n⏱️ Tiempo total: {elapsed:.2f} segundos")
    
    return engine

# Ejecutar
engine = await run_backtest()

## 4. Análisis de Resultados

In [ ]:
# Leer y mostrar métricas del audit
import re

def analyze_audit(audit_path='backtest_audit.md'):
    with open(audit_path, 'r') as f:
        content = f.read()
    
    # Extraer métricas
    progress_lines = re.findall(r'Progreso.*Balance: ([\d.]+).*Equity: ([\d.]+).*skipped', content)
    
    if progress_lines:
        last = progress_lines[-1]
        print(f"💰 Balance Final: {float(last[0]):,.2f} EUR")
        print(f"📊 Equity Final: {float(last[1]):,.2f} EUR")
    
    # Contar TPs
    main_tps = len(re.findall(r'Main TP detected', content))
    recovery_tps = len(re.findall(r'FULLY RECOVERED', content))
    
    print(f"\n🎯 Main TPs: {main_tps}")
    print(f"🔄 Recovery TPs: {recovery_tps}")
    
    # Calcular skip %
    skip_match = re.findall(r'([\d.]+)% skipped', content)
    if skip_match:
        avg_skip = sum(float(s) for s in skip_match) / len(skip_match)
        print(f"\n⚡ Ticks Skipped (avg): {avg_skip:.1f}%")

analyze_audit()

In [ ]:
# Visualizar Equity Curve
import matplotlib.pyplot as plt
import re

def plot_equity_curve(audit_path='backtest_audit.md'):
    with open(audit_path, 'r') as f:
        content = f.read()
    
    # Extraer balance de cada progreso
    matches = re.findall(r'Balance: ([\d.]+)', content)
    balances = [float(m) for m in matches]
    
    if not balances:
        print("No se encontraron datos de balance")
        return
    
    plt.figure(figsize=(12, 6))
    plt.plot(balances, 'g-', linewidth=2)
    plt.axhline(y=INITIAL_BALANCE, color='gray', linestyle='--', alpha=0.5)
    plt.title('📈 WSPlumber Equity Curve', fontsize=14)
    plt.xlabel('Checkpoint')
    plt.ylabel('Balance (EUR)')
    plt.grid(True, alpha=0.3)
    plt.fill_between(range(len(balances)), balances, INITIAL_BALANCE, 
                     where=[b > INITIAL_BALANCE for b in balances], alpha=0.3, color='green')
    plt.fill_between(range(len(balances)), balances, INITIAL_BALANCE,
                     where=[b <= INITIAL_BALANCE for b in balances], alpha=0.3, color='red')
    plt.show()
    
    # Stats
    print(f"\n📊 Estadísticas:")
    print(f"   Balance Inicial: {INITIAL_BALANCE:,.2f} EUR")
    print(f"   Balance Final: {balances[-1]:,.2f} EUR")
    print(f"   Profit: {balances[-1] - INITIAL_BALANCE:+,.2f} EUR ({((balances[-1]/INITIAL_BALANCE)-1)*100:+.2f}%)")
    print(f"   Max Balance: {max(balances):,.2f} EUR")
    print(f"   Min Balance: {min(balances):,.2f} EUR")

plot_equity_curve()

## 5. Descargar Resultados

In [ ]:
# Descargar el archivo de audit
from google.colab import files

files.download('backtest_audit.md')
print("📥 Archivo de audit descargado")

---

## 📌 Notas

- **Skip de Ticks:** El backtest solo procesa ticks que pueden activar órdenes o TPs
- **Tiempo estimado:** ~100k ticks en ~2-3 minutos con la optimización
- **Para más datos:** Ajusta `MAX_TICKS = None` para procesar todo el archivo